In [1]:
#标记清除：
# 容器：所有可以引用其他对象的对象都叫容器，只有容器之间会存在循环引用，非容器对象可以引用，但不会出现循环引用
# 1、用双向列表存储所有容器对象，初始化所有容器的gc_ref值，初始化为其对应的引用计数值
# 2、遍历所有的容器对象，将其所有引用的对象的gc_ref值进行减1，如A引用B和C，则对B和C的gc_ref值减一
# 3、将所有gc_ref值大于0的容器放到一个集合里，大于0，则表明还存在非容器对象对其引用，故而不能回收
# 4、将集合中所有容器所引用的对象也收集起来，因为还存在引用，故而不能回收
# 5、将剩余的容器进行垃圾回收操作，表明都是循环引用，可以回收

In [2]:
#首先需要自己实现一个双向列表，基本功能包括：
#左添加：append_left(node)
#右添加：append_right(node)
#左删除：remove_left()
#右删除：remove_right()
#删除指定节点：delete_node(node)
#获取长度：get_length()
#清除所有：clear()
#做完后发现，其实功能不需要那么多，append_right(node)，get_length()，clear()三个就足够了。

In [3]:
class Node:
    def __init__(self,node_value=None,ref_value=None):
        self.pre_node = None
        self.next_node = None
        self.node_value = node_value   #int
        self.ref_value = ref_value     #list
class myLinkedList:
    def __init__(self):
        self.head = Node()
        self.tail = self.head
        self.head.next_node = self.tail
        self.tail.pre_node = self.head
        self.length = 0
    def append_left(self,node):
        if self.length == 0:
            self.head.node_value = node.node_value   
            self.head.ref_value = node.ref_value     
        else:
            self.head.pre_node = node
            node.next_node = self.head
            self.head = node
        self.length += 1
    def append_right(self,node):
        if self.length == 0:
            self.head.node_value = node.node_value   
            self.head.ref_value = node.ref_value     
        else:
            self.tail.next_node = node
            node.pre_node = self.tail
            self.tail = node
        self.length += 1
    def remove_left(self):
        removed_head = self.head
        self.head = self.head.next_node
        self.length -= 1
        return removed_head
    def remove_right(self):
        removed_tail = self.tail
        self.tail = self.tail.pre_node
        self.length -= 1
        return removed_tail
    def delete_node(self,node):
        temp = self.head
        for idx in range(self.lenght):
            if temp.node_value == node.node_value:
                target = temp
                break
            temp = temp.next
        if target:
            target.pre_node.next = target.next
            target.next.pre_node = target.pre_node
            self.length -= 1
        else:
            print("No such node")
    def get_length(self):
        return self.length
    def clear(self):
        import gc
        temp = self.head
        while temp.next_node:
            next_node = temp.next_node
            del temp
            temp = next_node
        del temp
        gc.collect()
        self.length = 0

In [4]:
#初始化引用情况
reference_dict = {1:[2],2:[1],3:[4],4:[5],5:[3],6:[7,8],7:[8,9],8:[9,10],9:[10]}
#这里表示容器1引用了容器2，容器2引用了容器1，有相互引用，3，4，5之间有相互引用，6，7，8，9，10之间不存在相互引用
count_dict = {1:1,2:1,3:1,4:1,5:1,6:0,7:2,8:2,9:2,10:3}
#这里是每个容器对应的引用计数，如1被一个引用，2被一个引用，6被0个引用，7被2个引用，一个是容器6，另一个是非容器对象
#所以应该清扫1，2，3，4，5这五个,1，2存在循环引用，3，4，5存在循环引用。再加一个6，因为6虽然引用了7，8，但不存在容器引用6，故而不再需要了。
#故而最终应该只剩下7，8，9，10四个
dire_list = myLinkedList()
for key,value in reference_dict.items():
    node = Node(node_value = key,ref_value = value)
    dire_list.append_right(node)
print(dire_list.get_length())

9


In [5]:
dc_ref_dict = count_dict
head = dire_list.head
print(head.node_value)
while head:
    ref_list = head.ref_value
    #print(ref_list)
    for value in ref_list:
        dc_ref_dict.update({value:dc_ref_dict.get(value)-1})
    head = head.next_node
#检查是否有大于1的容器,有则先拿出来，这里会将7先拿出来。
sortedList = []
head = dire_list.head
while head:
    node_value = head.node_value
    if dc_ref_dict[node_value] > 0:
        sortedList.append(head)
    head = head.next_node
#print(len(sortedList))
sortedNodes = []

#将保存在列表里的容器所对应的引用容器都拿出来
for node in sortedList:
    node_value = node.node_value
    ref_list = node.ref_value
    sortedNodes.append(node_value)
    sortedNodes.extend(ref_list)

print(sortedNodes) #7引用8和9，故而这里会存在7，8，9
    
while True:
    before_length = len(sortedNodes)
    for each in sortedNodes:
        refrences = reference_dict.get(each)
        if refrences:
            #print(refrences)
            [sortedNodes.append(ref) for ref in refrences if ref not in set(sortedNodes)]
    after_length = len(sortedNodes)    
    if before_length == after_length:
        break

print(sortedNodes)  #此时会发现7，8，9，10都保留了下来
print(len(sortedNodes))

dire_list.clear() #把双向列表的内容都清除即可

1
[7, 8, 9]
[7, 8, 9, 10]
4
